In [ ]:
import sympy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import dill

In [ ]:
t, x, y, h1, h2 = sympy.symbols("t x y h1 h2", real=True)

In [ ]:
phi_0 = sympy.Piecewise(
    ((1 + t) ** 2 * (1 - 2 * t), (t >= -1) & (t <= 0)),
    ((1 - t) ** 2 * (1 + 2 * t), (t >= 0) & (t <= 1)),
    (0, True),
)

phi_1 = sympy.Piecewise(
    ((1 + t) ** 2 * t, (t >= -1) & (t <= 0)),
    ((1 - t) ** 2 * t, (t >= 0) & (t <= 1)),
    (0, True),
)

In [ ]:
origin_choices = [
                    [phi_0, phi_0, 1, 1], 
                    [phi_1, phi_0, h1, 1], 
                    [phi_0, phi_1, 1, h2], 
                    [phi_1, phi_1, h1, h2],
                ]
choices = [
                    [phi_0, phi_0, 1, 1], 
                    [phi_1, phi_0, h1, 1], 
                    [phi_0, phi_1, 1, h2], 
                    [phi_1, phi_1, h1, h2]
            ]

matrix = np.zeros((4, 4, 3, 3)).tolist()
right_part = np.zeros((4, 4, 3, 3)).tolist()

for i, [origin_base_X, origin_base_Y, origin_Jx, origin_Jy] in enumerate(origin_choices):
    for j, [base_X, base_Y, Jx, Jy] in enumerate(choices):
        for k, y_offset in enumerate([-1, +0, +1]):
            for l, x_offset in enumerate([-1, +0, +1]):

                xx_x = origin_base_X.diff(t, t) * base_X.subs(t, t-x_offset).diff(t, t) / h1**4 * origin_Jx * Jx
                xx_y = origin_base_Y * base_Y.subs(t, t-y_offset) * origin_Jy * Jy

                xy_x = origin_base_X.diff(t) * base_X.subs(t, t-x_offset).diff(t) / h1**2 * origin_Jx * Jx
                xy_y = origin_base_Y.diff(t) * base_Y.subs(t, t-y_offset).diff(t) / h2**2 * origin_Jy * Jy

                yy_x = origin_base_X * base_X.subs(t, t-x_offset) * origin_Jx * Jx
                yy_y = origin_base_Y.diff(t, t) * base_Y.subs(t, t-y_offset).diff(t, t) / h2**4 * origin_Jy * Jy

                a = sympy.integrate(xx_x, (t, -1, +1)) * sympy.integrate(xx_y, (t, -1, +1)) * h1*h2
                b = sympy.integrate(xy_x, (t, -1, +1)) * sympy.integrate(xy_y, (t, -1, +1)) * h1*h2
                c = sympy.integrate(yy_x, (t, -1, +1)) * sympy.integrate(yy_y, (t, -1, +1)) * h1*h2
                
                
                d_x = origin_base_X * base_X.subs(t, t-x_offset) * origin_Jx * Jx
                d_y = origin_base_Y * base_Y.subs(t, t-y_offset) * origin_Jy * Jy
                
                d = sympy.integrate(d_x, (t, -1, +1)) * sympy.integrate(d_y, (t, -1, +1)) * h1*h2
                
                
                matrix[i][j][k][l] = a+2*b+c
                right_part[i][j][k][l] = d

In [ ]:
dill.settings["recurse"] = True
dill.dump(sympy.lambdify([h1, h2], sympy.Array(np.array(matrix))), open("../calculations/hermite_cubic_biharmonic_matrix_integral_values", "wb"))
dill.dump(sympy.lambdify([h1, h2], sympy.Array(np.array(right_part))), open("../calculations/hermite_cubic_biharmonic_matrix_right_values", "wb"))